<a href="https://colab.research.google.com/github/Adiramcharan/leaf-disease-detection-using-SVM-and-ResNet50/blob/main/RBL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()  # Upload the kaggle.json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install tensorflow scikit-learn matplotlib

Saving kaggle.json to kaggle.json


In [ ]:
! kaggle datasets download -d vipoooool/new-plant-diseases-dataset

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
 99% 2.68G/2.70G [00:42<00:00, 160MB/s]
100% 2.70G/2.70G [00:42<00:00, 67.5MB/s]


In [ ]:
import zipfile
import os

# Unzip the dataset
with zipfile.ZipFile('new-plant-diseases-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset-folder')

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
def extract_features_batch(image_paths, model, batch_size=32):
    all_features = []
    for start in range(0, len(image_paths), batch_size):
        end = min(start + batch_size, len(image_paths))
        batch_paths = image_paths[start:end]
        batch_images = []

        for img_path in batch_paths:
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is not None:
                img = cv2.resize(img, (224, 224))
                img = preprocess_input(img)
                batch_images.append(img)

        batch_images = np.array(batch_images)
        batch_features = model.predict(batch_images)
        batch_features = batch_features.reshape(batch_features.shape[0], -1)
        all_features.extend(batch_features)

    return np.array(all_features)

In [ ]:
def get_image_paths(directory):
    image_paths = []
    labels = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for file in os.listdir(label_dir):
            img_path = os.path.join(label_dir, file)
            image_paths.append(img_path)
            labels.append(label)

    return image_paths, labels

# Set the path to your dataset
train_dir = '/content/dataset-folder/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_dir = '/content/dataset-folder/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

train_image_paths, train_labels = get_image_paths(train_dir)
valid_image_paths, valid_labels = get_image_paths(valid_dir)

# Encode labels
label_dict = {label: idx for idx, label in enumerate(np.unique(train_labels))}
train_labels_encoded = np.array([label_dict[label] for label in train_labels])
valid_labels_encoded = np.array([label_dict[label] for label in valid_labels])

In [ ]:
# Load ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Check the model summary to find the correct layer name
base_model.summary()

# Alternatively, use the output just before the classification layers
model_output = base_model.output
model_output = tf.keras.layers.GlobalAveragePooling2D()(model_output)
model_avg_pool = Model(inputs=base_model.input, outputs=model_output)

94765736/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                      

In [ ]:
# Extract features in batches
train_features_avg_pool = extract_features_batch(train_image_paths, model_avg_pool)
valid_features_avg_pool = extract_features_batch(valid_image_paths, model_avg_pool)


1/1 [==============================] - 0s 49ms/step


In [ ]:
# Standardize the features
scaler_avg_pool = StandardScaler()
train_features_avg_pool_scaled = scaler_avg_pool.fit_transform(train_features_avg_pool)
valid_features_avg_pool_scaled = scaler_avg_pool.transform(valid_features_avg_pool)

# Train SVM model
svm_avg_pool = SVC(kernel='linear')
svm_avg_pool.fit(train_features_avg_pool_scaled, train_labels_encoded)

# Make predictions
train_preds_avg_pool = svm_avg_pool.predict(train_features_avg_pool_scaled)
valid_preds_avg_pool = svm_avg_pool.predict(valid_features_avg_pool_scaled)

# Evaluate the model
train_accuracy_avg_pool = accuracy_score(train_labels_encoded, train_preds_avg_pool)
valid_accuracy_avg_pool = accuracy_score(valid_labels_encoded, valid_preds_avg_pool)

print(f'Training Accuracy (avg_pool): {train_accuracy_avg_pool}')
print(f'Validation Accuracy (avg_pool): {valid_accuracy_avg_pool}')
print('Classification Report (avg_pool):')
print(classification_report(valid_labels_encoded, valid_preds_avg_pool))

In [ ]:
import pickle

# Save the model to disk
with open('svm_resnet_model.pkl', 'wb') as file:
    pickle.dump(svm_avg_pool, file)

# Load the saved model
with open('svm_resnet_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Now you can use loaded_model to make predictions or further evaluate

In [ ]:
# Plot confusion matrix
conf_matrix_avg_pool = confusion_matrix(valid_labels_encoded, valid_preds_avg_pool)
plt.figure(figsize=(10, 7))
plt.title('Confusion Matrix (avg_pool)')
plt.imshow(conf_matrix_avg_pool, interpolation='nearest', cmap='Blues')
plt.colorbar()
tick_marks = np.arange(len(label_dict))
plt.xticks(tick_marks, label_dict.keys(), rotation=45)
plt.yticks(tick_marks, label_dict.keys())
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()